<h2> Data source for Traffic Based Automated Scheduling </h2><br>
<h4> Author: Dhananjay Kumar: dhananjay.kumar@spsmail.cuny.edu </h4>

The data for this project comes from following source:

Sales transactions (download .csv file)

The Sales Jan 2009 file contains some “sanitized” sales transactions during the month of January. Below is a sample of a report built in just a couple of minutes using the Blank Canvas app. These 997 transactions are easily summarized and filtered by transaction date, payment type, country, city, and geography. Note that these records are already geocoded, so you can use the existing latitude/longitude in the file.

Link of the website: https://support.spatialkey.com/spatialkey-sample-csv-data/

Link of the csv file: https://samplecsvs.s3.amazonaws.com/SalesJan2009.csv


We download the above data and upload the downloaded data into PostgreSQL database hosted on AWS (AWS RDS). Below is how the data looks

In [2]:
import sqlalchemy 
import psycopg2 
import random
import pandas as pd
import datetime as dt
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns 
sns.set(rc={'figure.figsize':(15,5)})

In [3]:
from sqlalchemy import create_engine
# Postgres username, password, and database name
POSTGRES_ADDRESS = '' ## INSERT YOUR DB ADDRESS IF IT'S NOT ON PANOPLY
POSTGRES_PORT = '5432' # Postgre Port no. by default it is 5432
POSTGRES_DBNAME='' # DB Name
POSTGRES_USERNAME = '' ## CHANGE THIS TO YOUR POSTGRES USERNAME
POSTGRES_PASSWORD = '' ## CHANGE THIS TO YOUR POSTGRES PASSWORD 
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}').format(username=POSTGRES_USERNAME,password=POSTGRES_PASSWORD,ipaddress=POSTGRES_ADDRESS,port=POSTGRES_PORT,dbname=POSTGRES_DBNAME)
# Create the connection
cnx = create_engine(postgres_str) 

In [4]:
df= pd.read_sql_query("""
select
 cast(cast(left(transaction_date, 7) as date) +
                     cast(right(transaction_date, 5) as time) as timestamp)     as ActualTS,
       cast(left(transaction_date, 7) as date)           as Date,
       cast(right(transaction_date, 5) as time)          as ActualTime,
       cast(date_trunc('hours', cast(cast(left(transaction_date, 7) as date) +
                                        cast(right(transaction_date, 5) as time) as timestamp)) as time) as round_off_time,
       name,city,state,country
from sales_transaction;
""", cnx)

In [5]:
df.head()

,actualts,date,actualtime,round_off_time,name,city,state,country
0,2009-01-02 06:17:00,2009-01-02,06:17:00,06:00:00,carolina,Basildon,England,United Kingdom
1,2009-01-02 04:53:00,2009-01-02,04:53:00,04:00:00,Betina,Parkville,MO,United States
2,2009-01-02 13:08:00,2009-01-02,13:08:00,13:00:00,Federica e Andrea,Astoria,OR,United States
3,2009-01-03 14:44:00,2009-01-03,14:44:00,14:00:00,Gouya,Echuca,Victoria,Australia
4,2009-01-04 12:56:00,2009-01-04,12:56:00,12:00:00,Gerd W,Cahaba Heights,AL,United States


Based on above data we need to create two more dataset:

1. Customer Traffic : This is the data which will tell us how much traffic does a retail store attracts.
2. Employee: This dataset will tell us how many Employees are present at any given time within a store

Below is the source code for generating these two datasets

In [6]:
# Generate synthetic Customer Traffic Data !!! Don't Run unless new dataset needed !!!
#trafficList=[]
#for index, row in df.iterrows():
#    trafficList.append([row['round_off_time'],row['date'],row['city'],row['state'],row['country'],random.randint(1,5)])
#df3=pd.DataFrame(trafficList,columns=['Time','Date','City','State','Country','Traffic'])
#df3.to_csv("traffic.csv", index=False) 

In [7]:
# Generate synthetic Employee In Store Data !!! Don't Run unless new dataset needed !!!
#trafficList=[]
#for index, row in df.iterrows():
#    trafficList.append([row['round_off_time'],row['date'],row['city'],row['state'],row['country'],random.randint(2,5)])
#df4=pd.DataFrame(trafficList,columns=['Time','Date','City','State','Country','Employee_In_Store'])
#df4.to_csv("Employee.csv", index=False) 

The above two code chunk will generate two files "Traffic.csv" and "Employee.csv". Using these two files two new tables "Traffic" and "Employee" on postgre database on AWS
